In [1]:
import sys
import neal
import greedy
import tabu
import time
import numpy as np

from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../code/'))
from experiment import Experiment
from table import Table
from visualisation import Figure

# Load the data

In [2]:
data_folder = '../data/Travelling Salesman Problem/Small/'

file_names = ['bayg29.npz', 'bays29.npz', 'dantzig42.npz',
             'fri26.npz', 'gr17.npz', 'gr21.npz', 'gr24.npz']

loaded_files = [np.load(data_folder + file) for file in file_names]

obj_qubos = [i['cost_function_qubo'] for i in loaded_files]
obj_constants = [i['cost_function_constant'] for i in loaded_files]
con_qubos = [i['constraint_function_qubo'] for i in loaded_files]
con_constants = [i['constraint_function_constant'] for i in loaded_files]

# Prepare the data

Here we already get the qubos and constraints. So we only need to calculate penalties and get the full QUBOs.

In [3]:
minimisation = True # This is a minimisation problem
QUBOs, penalties = Experiment.data_prep_light(obj_qubos, con_qubos, 'Verma&Lewis', minimisation)
qubo_sizes = [max(qubo, key=tuple)[0] + 1 for qubo in QUBOs]

# Run experiments

In [4]:
# The number of samples we want the sampler to return
repeats = 60

## Greedy

In [5]:
greedy_sampler = greedy.SteepestDescentSampler()
greedy_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                     greedy_sampler, repeats, num_reads=180)

100.0 %


## Simulated Annealing

In [6]:
sa_sampler = neal.SimulatedAnnealingSampler()
sa_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                 sa_sampler, repeats, num_reads=30)

100.0 %


## Tabu

In [7]:
tabu_sampler = tabu.TabuSampler()
tabu_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                   tabu_sampler, repeats, timeout=4000)

100.0 %


# Record the results

In [8]:
greedy_results = Table.record_results(greedy_runs, qubo_sizes, penalties, repeats, minimisation)
sa_results = Table.record_results(sa_runs, qubo_sizes, penalties, repeats, minimisation)
tabu_results = Table.record_results(tabu_runs, qubo_sizes, penalties, repeats, minimisation)

# Display the first repetition table
rep = 0
Table.display_side_by_side(greedy_results[rep], sa_results[rep], tabu_results[rep], titles=['Greedy', 'SA', 'Tabu'])

,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,784,12353,2244,0,2244
1,784,16945,2909,0,2909
2,1681,9877,1186,0,1186
3,625,9446,1359,0,1359
4,256,15329,2440,0,2440
5,400,21685,3808,0,3808
6,529,10127,1751,0,1751
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,784,12353,3749,0,3749
1,784,16945,4879,0,4879


# Explore the results

In [9]:
# Show total energies of all tries in all problems in a single df
energies_greedy = Table.columns_to_table(greedy_results, 'Energy (minimisation)')
energies_sa = Table.columns_to_table(sa_results, 'Energy (minimisation)')
energies_tabu = Table.columns_to_table(tabu_results, 'Energy (minimisation)')

energies_tabu

,Energy (minimisation) 0,Energy (minimisation) 1,Energy (minimisation) 2,Energy (minimisation) 3,Energy (minimisation) 4,Energy (minimisation) 5,Energy (minimisation) 6,Energy (minimisation) 7,Energy (minimisation) 8,Energy (minimisation) 9,...,Energy (minimisation) 50,Energy (minimisation) 51,Energy (minimisation) 52,Energy (minimisation) 53,Energy (minimisation) 54,Energy (minimisation) 55,Energy (minimisation) 56,Energy (minimisation) 57,Energy (minimisation) 58,Energy (minimisation) 59
0,1756,1757,1765,1789,1750,1783,1750,1814,1753,1813,...,1770,1807,1743,1823,1831,1792,1795,1788,1833,1791
1,2187,2191,2222,2231,2104,2239,2203,2228,2212,2246,...,2163,2166,2179,2195,2222,2184,2214,2246,2189,2199
2,755,699,819,768,775,763,699,699,751,767,...,756,699,782,811,699,699,699,699,699,704
3,1076,993,1035,1057,1023,1029,1071,1023,989,1009,...,1025,1031,1020,1056,1030,1012,1034,1035,997,1045
4,2176,2171,2174,2152,2098,2146,2168,2147,2128,2157,...,2151,2182,2119,2170,2098,2176,2100,2120,2161,2090
5,2979,2832,2910,3015,2934,2988,2956,3005,2853,2916,...,2807,2807,2816,2891,2977,2870,2818,2898,2839,2756
6,1400,1374,1358,1385,1347,1380,1361,1388,1344,1357,...,1419,1352,1322,1318,1358,1423,1371,1371,1379,1346


In [10]:
# Show number of broken constraints of all tries in all problems in a single df
broken_constraints_greedy = Table.columns_to_table(greedy_results, 'Broken Constraints')
broken_constraints_sa = Table.columns_to_table(sa_results, 'Broken Constraints')
broken_constraints_tabu = Table.columns_to_table(tabu_results, 'Broken Constraints')

broken_constraints_greedy

,Broken Constraints 0,Broken Constraints 1,Broken Constraints 2,Broken Constraints 3,Broken Constraints 4,Broken Constraints 5,Broken Constraints 6,Broken Constraints 7,Broken Constraints 8,Broken Constraints 9,...,Broken Constraints 50,Broken Constraints 51,Broken Constraints 52,Broken Constraints 53,Broken Constraints 54,Broken Constraints 55,Broken Constraints 56,Broken Constraints 57,Broken Constraints 58,Broken Constraints 59
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Show feasible solutions
feasible_full_greedy = Table.feasibility_table(greedy_results)
feasible_full_sa = Table.feasibility_table(sa_results)
feasible_full_tabu = Table.feasibility_table(tabu_results)

feasible_full_tabu

,Feasible 0,Feasible 1,Feasible 2,Feasible 3,Feasible 4,Feasible 5,Feasible 6,Feasible 7,Feasible 8,Feasible 9,...,Feasible 50,Feasible 51,Feasible 52,Feasible 53,Feasible 54,Feasible 55,Feasible 56,Feasible 57,Feasible 58,Feasible 59
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


This is the final and the most important table. It displays the number of feasible solutions achieved for every problem, the feasibility rate (feasible_soultions/total_solutions), the mean energy of the solutions and the standard deviation.

In [12]:
# Calculate number of feasible solutions with mean and SD (in all runs)
feasible_greedy = Table.feasibility_statistic(greedy_results)
feasible_sa = Table.feasibility_statistic(sa_results)
feasible_tabu = Table.feasibility_statistic(tabu_results)

# Display the table
Table.display_side_by_side(feasible_greedy, feasible_sa, feasible_tabu, titles=['Greedy', 'SA', 'Tabu'])

,Feasible,Feasibility rate,Energy mean,Energy SD
0,60.0,1.0,2257.766667,92.100418
1,60.0,1.0,2861.400000,98.105650
2,60.0,1.0,1201.733333,49.259213
3,60.0,1.0,1294.783333,42.583156
4,60.0,1.0,2471.316667,91.223602
5,60.0,1.0,3801.750000,164.459631
6,60.0,1.0,1744.033333,66.583442
Total,420.0,7.0,15632.783333,604.315112
Mean,60.0,1.0,2233.254762,86.330730
SD,0.0,0.0,921.603742,40.780115


# Save results

In [13]:
data_folder = '../Data/Produced/Travelling Salesman Problem/Small/'
broken_constraints_greedy.to_pickle(data_folder + 'vlewis_tsp_greedy_broken_constraints.pkl')
broken_constraints_sa.to_pickle(data_folder + 'vlewis_tsp_sa_broken_constraints.pkl')
broken_constraints_tabu.to_pickle(data_folder + 'vlewis_tsp_tabu_broken_constraints.pkl')

feasible_greedy.to_pickle(data_folder + 'vlewis_tsp_greedy_feasible.pkl')
feasible_sa.to_pickle(data_folder + 'vlewis_tsp_sa_feasible.pkl')
feasible_tabu.to_pickle(data_folder + 'vlewis_tsp_tabu_feasible.pkl')